## This Notebook is to read in the model and then translate from English to Spanish

In [2]:
# Standard imports
import pandas as pd
import numpy as np
import re
import pickle
import string
import json

# Visualization library
import seaborn as sns
import matplotlib.pyplot as plt

# NLP library
from nltk.tokenize import word_tokenize
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.layers import Input, LSTM, Dense, Masking
from tensorflow.keras.models import Model
from keras.backend import manual_variable_initialization

manual_variable_initialization(True)

In [3]:
from numpy.random import seed
seed(42)
tf.random.set_seed(42)


In [4]:
# Encoder training setup
num_encoder_tokens = 4428
num_decoder_tokens = 7675

max_encoder_seq_length = 7
max_decoder_seq_length = 16

latent_dim = 256

In [5]:
import pickle

with open('./data/rtfd.p', 'rb') as fp:
    reverse_target_features_dict = pickle.load(fp)
    
with open('./data/tfd.p', 'rb') as fp:
    target_features_dict = pickle.load(fp)
    
with open('./data/rifd.p', 'rb') as fp:
    reverse_input_features_dict = pickle.load(fp)

with open('./data/ifd.p', 'rb') as fp:
    input_features_dict = pickle.load(fp)

## Next few cells are to confirm that dictionary and reverse dictionaries are working as expected

In [6]:
# Python 3
first2pairs = {k: input_features_dict[k] for k in list(input_features_dict)[:2]}
first2pairs

{'a': 0, 'aback': 1}

In [7]:
# Python 3
first2pairs = {k: reverse_input_features_dict[k] for k in list(reverse_input_features_dict)[:2]}
first2pairs

{0: 'a', 1: 'aback'}

In [8]:
# Python 3
first2pairs = {k: target_features_dict[k] for k in list(target_features_dict)[:10]}
first2pairs

{'<END>': 0,
 '<START>': 1,
 'a': 2,
 'abajo': 3,
 'abandona': 4,
 'abandonada': 5,
 'abandonado': 6,
 'abandonamos': 7,
 'abandonar': 8,
 'abandonaron': 9}

In [9]:
# Python 3
first2pairs = {k: reverse_target_features_dict[k] for k in list(reverse_target_features_dict)[:10]}
first2pairs

{0: '<END>',
 1: '<START>',
 2: 'a',
 3: 'abajo',
 4: 'abandona',
 5: 'abandonada',
 6: 'abandonado',
 7: 'abandonamos',
 8: 'abandonar',
 9: 'abandonaron'}

## Next block is to load the model that has been built and then functions needed for translation and then translate

In [12]:
#TRANSLATING UNSEEN TEXT
from tensorflow.keras.models import Model, load_model

#training_model = load_model('./models/')
training_model = load_model('training_model_v2.h5')
#training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#training_model = load_weights('tr_weights.h5')

encoder_inputs = training_model.input[0] #input1
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output #lstm1
encoder_states = [state_h_enc, state_c_enc]

encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256

decoder_inputs = training_model.input[1] #input2

decoder_state_input_hidden = Input(shape=(latent_dim),name="input_3")
decoder_state_input_cell = Input(shape=(latent_dim), name = "input_4")
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]

decoder_lstm = training_model.layers[3]

decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]

decoder_dense = training_model.layers[4]

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [13]:
## This function takes in the string to be translated and converts into the matrix that is the intput into the LSTM model

def string_to_matrix(user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for timestep, token in enumerate(tokens):
        if token in input_features_dict:
            user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix

In [14]:
## This function is used to pass the sentence to be translates, makes other function calls and then translates the sentence
def decode_sequence(test_input):
   
    # Encode the input as state vectors.
    states_value = encoder_model.predict(string_to_matrix(test_input))

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first token of target sequence with the start token.
    target_seq[0, 0, target_features_dict['<START>']] = 1.

    # Sampling loop for a batch of sequences
    decoded_sentence = ''

    stop_condition = False
    while not stop_condition:
        # Run the decoder model to get possible output tokens (with probabilities) & states
        output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)

        # Choose token with highest probability and append it to decoded sentence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token

        # Exit condition: either hit max length or find stop token.
        if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [hidden_state, cell_state]

    return decoded_sentence

In [15]:
# Example decoded sentence
decode_sequence('how are you')

IndexError: index 5024 is out of bounds for axis 2 with size 4428

In [ ]:
# Example decoded sentence
decode_sequence('it is hot today')

In [ ]:
# Example decoded sentence
decode_sequence('what is your name')